# Co-Attention Siamese Model Training

### Importing Libraries

In [ ]:
!pip install pandas==2.2.3
!pip install numpy==1.26.4
!pip install optuna==4.1.0
!pip install regex==2024.9.11
!pip install tensorflow==2.17.0
!pip install scikit-learn==1.5.2
!pip install sentence_transformers==3.4.1

Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
  Using cached nvidia_cuda_nvrtc_cu12-12.4.127-py3-none-manylinux2014_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_runtime_cu12-12.4.127-py3-none-manylinux2014_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_cupti_cu12-12.4.127-py3-none-manylinux2014_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cudnn_cu12-9.1.0.70-py3-none-manylinux2014_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cublas_cu12-12.4.5.8-py3-none-manylinux2014_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cufft_cu12-11.2.1.3-py3-none-manylinux2014_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_curand_cu12-10.3.5.147-py3-none-manylinux2014_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cusolver_cu12-11.6.1.9-py3-none-manylinux2014_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cusparse_cu12-12.3.1.170-py3-none-manylinux201

In [2]:
import re
import optuna
import numpy as np
import pandas as pd
import tensorflow as tf

from tensorflow.keras import layers, Model
from sklearn.metrics import f1_score, classification_report, matthews_corrcoef
from sentence_transformers import SentenceTransformer

/home/chuongg3/.local/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2025-04-04 13:16:45.945124: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-04-04 13:16:45.971082: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2025-04-04 13:16:45.998993: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2025-04-04 13:16:46.007434: E

### Load Data

In [ ]:
# Load the training data
train_data_path = './training_data/ED/train.csv'
train_data = pd.read_csv(train_data_path)

In [ ]:
# Load the dev data
dev_data_path = './training_data/ED/dev.csv'
dev_data = pd.read_csv(dev_data_path)

#### Remove [ref] from Text

In [ ]:
# Remove instances of [ref] from the text
remove_ref = lambda x: re.sub(r'\[ref\]|\[ref|ref\]', '', x)

train_data['Claim'] = train_data['Claim'].apply(remove_ref)
train_data['Evidence'] = train_data['Evidence'].apply(remove_ref)
dev_data['Claim'] = dev_data['Claim'].apply(remove_ref)
dev_data['Evidence'] = dev_data['Evidence'].apply(remove_ref)

### Calculate Training Weights

In [16]:
zeroes = train_data[train_data['label'] == 0].shape[0]
ones = train_data[train_data['label'] == 1].shape[0]

print(f'Number of zeroes: {zeroes}')
print(f'Number of ones: {ones}')

Number of zeroes: 14088
Number of ones: 5269


In [17]:
# Add a column to training data for the weights
zero_weight = zeroes / (zeroes + ones)
one_weight = ones / (zeroes + ones)

train_data['weights'] = train_data['label'].apply(lambda x: zero_weight if x == 0 else one_weight)

### Sample Text Length

In [18]:
# Claim Maximum Length
claim_max_length = train_data['Claim'].apply(lambda x: len(str(x))).max()
print(f'Claim Maximum Length: {claim_max_length}')

Claim Maximum Length: 180


In [19]:
# Evidence Maximum Length
evidence_max_length = train_data['Evidence'].apply(lambda x: len(str(x))).max()
print(f'Evidence Maximum Length: {claim_max_length}')

Evidence Maximum Length: 180


### Encode Textual Inputs


In [ ]:
# Load the Encoder
encoder_model = SentenceTransformer("all-MiniLM-L6-v2")
# encoder_model = SentenceTransformer("multi-qa-mpnet-base-dot-v1")

# Encode the Training Set
claim_embeddings = encoder_model.encode(train_data['Claim'])
evidence_embeddings = encoder_model.encode(train_data['Evidence'])

# Encode the Dev Set
dev_claim_embeddings = encoder_model.encode(dev_data['Claim'])
dev_evidence_embeddings = encoder_model.encode(dev_data['Evidence'])

### Design the Model

In [ ]:
# Custom MultiHead Co-Attention layer
class MultiHeadCoAttention(layers.Layer):
    def __init__(self, attention_dim=128, num_heads=4, dropout_rate=0.1, **kwargs):
        super(MultiHeadCoAttention, self).__init__(**kwargs)
        self.attention_dim = attention_dim
        self.num_heads = num_heads
        self.head_dim = attention_dim // num_heads
        self.dropout_rate = dropout_rate
        
        # Ensure the attention dimension is divisible by the number of heads
        assert attention_dim % num_heads == 0, "Attention dimension must be divisible by number of heads"
        
    def build(self, input_shape):
        # Ensure we have two inputs
        assert isinstance(input_shape, list) and len(input_shape) == 2
        
        self.claim_dim = input_shape[0][-1]
        self.evidence_dim = input_shape[1][-1]
        
        # Claim projections
        self.claim_projection = layers.Dense(self.attention_dim * 3, use_bias=False)
        
        # Evidence projections
        self.evidence_projection = layers.Dense(self.attention_dim * 3, use_bias=False)
        
        # Output projections
        self.claim_output_projection = layers.Dense(self.claim_dim, use_bias=False)
        self.evidence_output_projection = layers.Dense(self.evidence_dim, use_bias=False)
        
        # Dropout layer
        self.dropout = layers.Dropout(self.dropout_rate)
        
        super(MultiHeadCoAttention, self).build(input_shape)
    
    def split_heads(self, x):
        """Split the last dimension into (num_heads, head_dim) with fixed batch size"""
        batch_size = tf.shape(x)[0]
        seq_len = tf.shape(x)[1]
        
        # Reshape with fixed dimensions where possible
        x = tf.reshape(x, [batch_size, seq_len, self.num_heads, self.head_dim])
        return tf.transpose(x, [0, 2, 1, 3])
    
    def call(self, inputs, training=None):
        # Unpack inputs
        claim, evidence = inputs

        # Get projection for claim (Q, K, V)
        claim_proj = self.claim_projection(claim)

        # Split claim projection into query, key, value
        claim_proj_split = tf.split(claim_proj, 3, axis=-1)
        claim_query, claim_key, claim_value = claim_proj_split

        # Get projection for evidence (Q, K, V)
        evidence_proj = self.evidence_projection(evidence)

        # Split evidence projection into query, key, value
        evidence_proj_split = tf.split(evidence_proj, 3, axis=-1)
        evidence_query, evidence_key, evidence_value = evidence_proj_split

        # Split heads
        claim_query_heads = self.split_heads(claim_query)
        claim_key_heads = self.split_heads(claim_key)
        claim_value_heads = self.split_heads(claim_value)

        evidence_query_heads = self.split_heads(evidence_query)
        evidence_key_heads = self.split_heads(evidence_key)
        evidence_value_heads = self.split_heads(evidence_value)

        # Claim attends to evidence with fixed scaling factor
        claim_evidence_scores = tf.matmul(claim_query_heads, evidence_key_heads, transpose_b=True)
        claim_evidence_scores = claim_evidence_scores / tf.sqrt(tf.cast(self.head_dim, tf.float32))

        claim_evidence_attention = tf.nn.softmax(claim_evidence_scores, axis=-1)
        claim_evidence_attention = self.dropout(claim_evidence_attention, training=training)

        claim_context = tf.matmul(claim_evidence_attention, evidence_value_heads)

        # Evidence attends to claim with fixed scaling factor
        evidence_claim_scores = tf.matmul(evidence_query_heads, claim_key_heads, transpose_b=True)
        evidence_claim_scores = evidence_claim_scores / tf.sqrt(tf.cast(self.head_dim, tf.float32))

        evidence_claim_attention = tf.nn.softmax(evidence_claim_scores, axis=-1)
        evidence_claim_attention = self.dropout(evidence_claim_attention, training=training)

        evidence_context = tf.matmul(evidence_claim_attention, claim_value_heads)

        # Combine heads and transpose back
        claim_context = tf.transpose(claim_context, [0, 2, 1, 3])
        evidence_context = tf.transpose(evidence_context, [0, 2, 1, 3])

        # Instead of reshaping to force seq_len=1, perform average pooling over the sequence dimension:
        claim_context = tf.reduce_mean(claim_context, axis=1, keepdims=True)
        evidence_context = tf.reduce_mean(evidence_context, axis=1, keepdims=True)

        # Reshape with fixed dimensions where possible
        batch_size = tf.shape(claim)[0]
        claim_context = tf.reshape(claim_context, [batch_size, 1, self.attention_dim])
        evidence_context = tf.reshape(evidence_context, [batch_size, 1, self.attention_dim])

        # Project back to original dimensions using Dense layers
        claim_output = self.claim_output_projection(claim_context)
        evidence_output = self.evidence_output_projection(evidence_context)

        return claim_output, evidence_output

In [12]:
def EncoderLayer(input_shape=(384, ), units = [512, 64], dropout=0.2):
    inputs = layers.Input(shape=input_shape)

    x = layers.Dense(units[0], activation="relu")(inputs)
    x = layers.Dropout(dropout)(x)
    x = layers.Dense(units[1], activation="relu")(x)

    return Model(inputs, x)

In [13]:
def get_model(input_shape=(384,) ,loss="binary_crossentropy", optimizer="adam", learning_rate=0.008, metrics=['accuracy', 'recall', 'precision'], units=[512, 64], attention_dim=128, dropout = [0.2, 0.5, 0.3], num_heads=4, ff_dim=64, attention_dropout=0.1):
    """
    Build a co-attention model for claim-evidence binary classification.
    
    Args:
        claim_dim: Dimension of the claim embedding
        evidence_dim: Dimension of the evidence embedding
        hidden_dim: Dimension of hidden layers
        
    Returns:
        A Keras Model instance
    """
    # Input layers
    claim_input = layers.Input(shape=input_shape, name="claim_input")
    evidence_input = layers.Input(shape=input_shape, name="evidence_input")

    # Siamese Encoder Layers
    encoder = EncoderLayer(input_shape=input_shape, units=units, dropout=dropout[0])
    claim_encoded = encoder(claim_input)
    evidence_encoded = encoder(evidence_input)

    # Reshape for co-attention (adding sequence dimension of 1)
    claim_reshaped = layers.Reshape((1, units[1]))(claim_encoded)
    evidence_reshaped = layers.Reshape((1, units[1]))(evidence_encoded)

    # Apply co-attention
    claim_attended, evidence_attended = MultiHeadCoAttention(attention_dim=attention_dim, num_heads=num_heads, dropout_rate=attention_dropout)(
        [claim_reshaped, evidence_reshaped]
    )

    claim_projected = layers.TimeDistributed(layers.Dense(ff_dim))(claim_attended)
    evidence_projected = layers.TimeDistributed(layers.Dense(ff_dim))(evidence_attended)

    # Flatten and concatenate
    claim_flat = layers.Reshape((ff_dim,))(claim_projected)
    evidence_flat = layers.Reshape((ff_dim,))(evidence_projected)

    merged = layers.Concatenate()([claim_flat, evidence_flat])

    # Classification layers
    dense1 = layers.Dense(ff_dim, activation="relu")(merged)
    bn1 = layers.BatchNormalization()(dense1)
    dropout1 = layers.Dropout(dropout[1])(bn1)
    dense2 = layers.Dense(ff_dim // 2, activation="relu")(dropout1)
    bn2 = layers.BatchNormalization()(dense2)
    dropout2 = layers.Dropout(dropout[2])(bn2)

    # Output layer
    output = layers.Dense(1, activation="sigmoid", name="output")(dropout2)

    # Compile model    
    model = Model(inputs=[claim_input, evidence_input], outputs=output)
    optimizer_instance = tf.keras.optimizers.get(optimizer)
    optimizer_instance.learning_rate = learning_rate
    model.compile(optimizer=optimizer_instance, loss=loss, metrics=metrics)

    return model

### Hyperparameter Training

In [ ]:
bestModel = None
bestModelScore = 0
bestThreshold = 0

# Define the objective function
def objective(trial):
    # Get a range of values
    epochs = trial.suggest_int('epochs', 5, 40)
    batch_size = trial.suggest_categorical('batch_size', [16, 32])
    learning_rate = trial.suggest_float('learning_rate', 1e-5, 1e-2)
    optimizer = trial.suggest_categorical('optimizer', ['adam'])

    # Model Specific Hyperparameters
    dropout0 = trial.suggest_float('dropout0', 0.1, 0.5)
    dropout1 = trial.suggest_float('dropout1', 0.1, 0.5)
    dropout2 = trial.suggest_float('dropout2', 0.1, 0.5)
    units = trial.suggest_categorical('units', [[512, 64], [512, 128], [512, 256], [1024, 64], [1024, 128], [1024, 256]])
    attention_dropout = trial.suggest_float('attention_dropout', 0.1, 0.5)
    attention_dim = trial.suggest_categorical('attention_dim', [32, 64, 128, 256])
    num_heads = trial.suggest_categorical('num_heads', [2, 4, 8, 16])
    ff_dim = trial.suggest_categorical('ff_dim', [64, 128, 256])

    dropout = [dropout0, dropout1, dropout2]

    print(f"""Testing Parameters:
    Epochs: {epochs}
    Dropout: {dropout}
    Units = {units}
    Attention Dropout: {attention_dropout}
    Batch Size: {batch_size}
    Learning Rate: {learning_rate}
    Optimizer: {optimizer}
    Attention Dimension: {attention_dim}
    Number of Heads: {num_heads}
    Feed Forward Dimension: {ff_dim}
""")

    metrics = ['accuracy', 'precision' , 'recall']
    training_set = (claim_embeddings, evidence_embeddings)
    validation_set = ((dev_claim_embeddings, dev_evidence_embeddings), dev_data['label'])

    early_stopping = tf.keras.callbacks.EarlyStopping(
        monitor='val_accuracy', min_delta=0, patience=10, verbose=0, mode='auto',
        baseline=None, restore_best_weights=True
    )

    # Create the model
    model = get_model(loss="binary_crossentropy",
                      optimizer='adam',
                      learning_rate=learning_rate,
                      metrics=metrics,
                      dropout=dropout,
                      units=units,
                      num_heads=num_heads,
                      ff_dim=ff_dim,
                      attention_dropout=attention_dropout,
                      attention_dim=attention_dim)

    model.fit(training_set, y=train_data['label'], epochs=epochs, validation_data=validation_set, callbacks=[early_stopping], sample_weight=train_data['weights'], batch_size=batch_size)

    # Get the predicted and true develoment labels
    print(" == Tuning Threshold ==")
    y_pred_prob = model.predict([dev_claim_embeddings, dev_evidence_embeddings])
    y_true = dev_data['label']

    # Threshold Tuning
    thresholds = np.arange(0.2, 0.7, 0.01)
    scores = [f1_score(y_true, (y_pred_prob > t).astype(int), average='weighted') for t in thresholds]
    best_threshold = thresholds[np.argmax(scores)]
    y_pred = (y_pred_prob > best_threshold).astype(int)

    # Calculate macro F1 Score
    report = classification_report(y_true=y_true, y_pred=y_pred, digits=10)
    print(report)
    macro_f1 = f1_score(y_true, y_pred, average='macro')
    weighted_f1 = f1_score(y_true, y_pred, average='weighted')

    # Update the best model
    global bestModel
    global bestModelScore
    if macro_f1 > bestModelScore:
        bestModel = model
        bestModelScore = macro_f1
        bestThreshold = best_threshold

    print(f"Best Threshold: {best_threshold}")
    print(f"Model Macro F1: {macro_f1}")
    print(f"Model Weighted F1: {weighted_f1}")

    return weighted_f1

# Create the Optuna study
study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=50)

# Print the best hyperparameters
if study.best_params:
    print("Best hyperparameters found:", study.best_params)
else:
    print("No valid hyperparameters found!")
model = bestModel

[I 2025-03-31 21:32:47,298] A new study created in memory with name: no-name-dfd559c3-32f8-48be-9069-ecee7398cbd1


Testing Parameters:
    Epochs: 13
    Dropout: [0.40766880520052284, 0.29254077431087755, 0.14040631580463483]
    Units = [512, 256]
    Attention Dropout: 0.15858138269517785
    Batch Size: 16
    Learning Rate: 0.0052136208076342
    Optimizer: adam
    Attention Dimension: 256
    Number of Heads: 8
    Feed Forward Dimension: 256

Epoch 1/13


/home/chuongg3/.local/lib/python3.10/site-packages/optuna/distributions.py:524: UserWarning: Choices for a categorical distribution should be a tuple of None, bool, int, float and str for persistent storage but contains [512, 64] which is of type list.
  warnings.warn(message)
/home/chuongg3/.local/lib/python3.10/site-packages/optuna/distributions.py:524: UserWarning: Choices for a categorical distribution should be a tuple of None, bool, int, float and str for persistent storage but contains [512, 128] which is of type list.
  warnings.warn(message)
/home/chuongg3/.local/lib/python3.10/site-packages/optuna/distributions.py:524: UserWarning: Choices for a categorical distribution should be a tuple of None, bool, int, float and str for persistent storage but contains [512, 256] which is of type list.
  warnings.warn(message)
/home/chuongg3/.local/lib/python3.10/site-packages/optuna/distributions.py:524: UserWarning: Choices for a categorical distribution should be a tuple of None, bool,

1345/1345 ━━━━━━━━━━━━━━━━━━━━ 12s 5ms/step - accuracy: 0.7168 - loss: 0.2369 - precision: 0.4754 - recall: 0.0948 - val_accuracy: 0.7479 - val_loss: 0.6160 - val_precision: 0.8724 - val_recall: 0.1043
Epoch 2/13
1345/1345 ━━━━━━━━━━━━━━━━━━━━ 4s 3ms/step - accuracy: 0.7607 - loss: 0.1761 - precision: 0.7480 - recall: 0.1622 - val_accuracy: 0.7442 - val_loss: 0.5581 - val_precision: 0.9559 - val_recall: 0.0793
Epoch 3/13
1345/1345 ━━━━━━━━━━━━━━━━━━━━ 4s 3ms/step - accuracy: 0.7762 - loss: 0.1633 - precision: 0.7913 - recall: 0.2374 - val_accuracy: 0.7776 - val_loss: 0.4470 - val_precision: 0.8797 - val_recall: 0.2274
Epoch 4/13
1345/1345 ━━━━━━━━━━━━━━━━━━━━ 4s 3ms/step - accuracy: 0.7747 - loss: 0.1609 - precision: 0.7955 - recall: 0.2424 - val_accuracy: 0.8014 - val_loss: 0.4297 - val_precision: 0.8513 - val_recall: 0.3421
Epoch 5/13
1345/1345 ━━━━━━━━━━━━━━━━━━━━ 4s 3ms/step - accuracy: 0.7951 - loss: 0.1553 - precision: 0.8345 - recall: 0.3154 - val_accuracy: 0.8068 - val_loss: 0.

[I 2025-03-31 21:33:47,804] Trial 0 finished with value: 0.8284567216981867 and parameters: {'epochs': 13, 'batch_size': 16, 'learning_rate': 0.0052136208076342, 'optimizer': 'adam', 'dropout0': 0.40766880520052284, 'dropout1': 0.29254077431087755, 'dropout2': 0.14040631580463483, 'units': [512, 256], 'attention_dropout': 0.15858138269517785, 'attention_dim': 256, 'num_heads': 8, 'ff_dim': 256}. Best is trial 0 with value: 0.8284567216981867.


...

              precision    recall  f1-score   support

           0  0.8824503311 0.8705086328 0.8764388067      4286
           1  0.6731448763 0.6969512195 0.6848412223      1640

    accuracy                      0.8224772190      5926
   macro avg  0.7777976037 0.7837299261 0.7806400145      5926
weighted avg  0.8245257706 0.8224772190 0.8234148380      5926

Best Threshold: 0.2700000000000001
Model Macro F1: 0.7806400144800802
Model Weighted F1: 0.8234148379930972
Testing Parameters:
    Epochs: 14
    Dropout: [0.3273556371865825, 0.3062260875317018, 0.25691544855200066]
    Units = [1024, 128]
    Attention Dropout: 0.11596540701304642
    Batch Size: 16
    Learning Rate: 0.00999765488720216
    Optimizer: adam
    Attention Dimension: 256
    Number of Heads: 4
    Feed Forward Dimension: 256

Epoch 1/14


/home/chuongg3/.local/lib/python3.10/site-packages/optuna/distributions.py:524: UserWarning: Choices for a categorical distribution should be a tuple of None, bool, int, float and str for persistent storage but contains [512, 64] which is of type list.
  warnings.warn(message)
/home/chuongg3/.local/lib/python3.10/site-packages/optuna/distributions.py:524: UserWarning: Choices for a categorical distribution should be a tuple of None, bool, int, float and str for persistent storage but contains [512, 128] which is of type list.
  warnings.warn(message)
/home/chuongg3/.local/lib/python3.10/site-packages/optuna/distributions.py:524: UserWarning: Choices for a categorical distribution should be a tuple of None, bool, int, float and str for persistent storage but contains [512, 256] which is of type list.
  warnings.warn(message)
/home/chuongg3/.local/lib/python3.10/site-packages/optuna/distributions.py:524: UserWarning: Choices for a categorical distribution should be a tuple of None, bool,

1345/1345 ━━━━━━━━━━━━━━━━━━━━ 12s 6ms/step - accuracy: 0.7264 - loss: 0.2271 - precision: 0.5241 - recall: 0.0834 - val_accuracy: 0.8112 - val_loss: 0.4526 - val_precision: 0.7911 - val_recall: 0.4317
Epoch 2/14
1345/1345 ━━━━━━━━━━━━━━━━━━━━ 4s 3ms/step - accuracy: 0.7759 - loss: 0.1678 - precision: 0.7969 - recall: 0.2377 - val_accuracy: 0.7519 - val_loss: 0.4453 - val_precision: 0.9722 - val_recall: 0.1067
Epoch 3/14
1345/1345 ━━━━━━━━━━━━━━━━━━━━ 4s 3ms/step - accuracy: 0.7765 - loss: 0.1629 - precision: 0.7765 - recall: 0.2352 - val_accuracy: 0.8078 - val_loss: 0.4373 - val_precision: 0.8224 - val_recall: 0.3896
Epoch 4/14
1345/1345 ━━━━━━━━━━━━━━━━━━━━ 4s 3ms/step - accuracy: 0.7849 - loss: 0.1598 - precision: 0.8127 - recall: 0.2820 - val_accuracy: 0.8014 - val_loss: 0.3994 - val_precision: 0.8852 - val_recall: 0.3244
Epoch 5/14
1345/1345 ━━━━━━━━━━━━━━━━━━━━ 5s 3ms/step - accuracy: 0.7943 - loss: 0.1518 - precision: 0.7933 - recall: 0.3070 - val_accuracy: 0.8247 - val_loss: 0.

[I 2025-03-31 22:13:40,688] Trial 49 finished with value: 0.8225316821536466 and parameters: {'epochs': 14, 'batch_size': 16, 'learning_rate': 0.00999765488720216, 'optimizer': 'adam', 'dropout0': 0.3273556371865825, 'dropout1': 0.3062260875317018, 'dropout2': 0.25691544855200066, 'units': [1024, 128], 'attention_dropout': 0.11596540701304642, 'attention_dim': 256, 'num_heads': 4, 'ff_dim': 256}. Best is trial 3 with value: 0.8428796728733234.


              precision    recall  f1-score   support

           0  0.8690313779 0.8917405506 0.8802395210      4286
           1  0.6963350785 0.6487804878 0.6717171717      1640

    accuracy                      0.8245021937      5926
   macro avg  0.7826832282 0.7702605192 0.7759783463      5926
weighted avg  0.8212382745 0.8245021937 0.8225316822      5926

Best Threshold: 0.3200000000000001
Model Macro F1: 0.7759783463376277
Model Weighted F1: 0.8225316821536466
Best hyperparameters found: {'epochs': 23, 'batch_size': 32, 'learning_rate': 0.007402024937971863, 'optimizer': 'adam', 'dropout0': 0.11597189185804023, 'dropout1': 0.1649105768518277, 'dropout2': 0.24244195005155006, 'units': [512, 64], 'attention_dropout': 0.10362259259272402, 'attention_dim': 256, 'num_heads': 4, 'ff_dim': 128}


### Evaluate the Model

In [17]:
eval = model.evaluate(x=[dev_claim_embeddings, dev_evidence_embeddings], y=dev_data['label'])

186/186 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.8292 - loss: 0.3837 - precision: 0.8145 - recall: 0.5087


In [18]:
accuracy = eval[1]
precision = eval[2]
recall = eval[3]
f1 = (2 * (precision * recall)) / (precision + recall)

print(f"Accuracy: {eval[1]}")
print(f"Precision: {eval[2]}")
print(f"Recall: {eval[3]}")
print(f"F1-Score: {f1}")

Accuracy: 0.8268646597862244
Precision: 0.8045634627342224
Recall: 0.4945122003555298
F1-Score: 0.6125377598654564


In [ ]:
# Get the predicted and true labels
y_pred = model.predict(x=[dev_claim_embeddings, dev_evidence_embeddings])
y_true = dev_data['label'].to_numpy()

186/186 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step


In [ ]:


# Threshold Tuning
thresholds = np.arange(0.2, 0.7, 0.01)
scores = [f1_score(y_true, (y_pred > t).astype(int), average='macro') for t in thresholds]
bestThreshold = thresholds[np.argmax(scores)]
print(f"Best Threshold: {bestThreshold}")

# Apply the best threshold to the predictions
y_pred_binary = (y_pred > bestThreshold).astype(int)

# Calculate the classification report
report = classification_report(y_true=y_true, y_pred=y_pred_binary, digits=10)
print(report)

Best Threshold: 0.34000000000000014
              precision    recall  f1-score   support

           0  0.8812868147 0.9076061596 0.8942528736      4286
           1  0.7380952381 0.6804878049 0.7081218274      1640

    accuracy                      0.8447519406      5926
   macro avg  0.8096910264 0.7940469822 0.8011873505      5926
weighted avg  0.8416590412 0.8447519406 0.8427417504      5926



In [ ]:


# Calculate Matthews Correlation Coefficient
mcc = matthews_corrcoef(y_true, y_pred_binary)
print(f"Matthews Correlation Coefficient: {mcc}")

In [ ]:
# model.save('CoAttentionSiameseDeepLearning.keras')